In [2]:
import pandas as pd
import maru

/home/waffdo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
df_titles = pd.read_csv("parsing_out_titles.csv") 

In [6]:
df_titles.drop('Unnamed: 0', 1, inplace = True)

In [10]:
df_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28026 entries, 0 to 28025
Data columns (total 2 columns):
doc_id    28026 non-null int64
title     27993 non-null object
dtypes: int64(1), object(1)
memory usage: 438.0+ KB


In [11]:
analyzer = maru.get_analyzer(tagger='crf', lemmatizer='pymorphy')

In [46]:
import nltk
import string
from nltk.corpus import stopwords

# nltk.download('punkt')
nltk.download('stopwords')
 
def tokenize_me(file_text):
    #firstly let's apply nltk tokenization
    tokens = nltk.word_tokenize(file_text)
 
    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]
 
    #deleting stop_words
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
    tokens = [i for i in tokens if ( i not in stop_words )]
 
    #cleaning words
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
 
    return tokens

[nltk_data] Downloading package stopwords to /home/waffdo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
titles_array = df_titles['title'].values

In [57]:
tokenize_me('мама мыла раму')
tokenized = [tokenize_me(str(sent)) for sent in titles_array]

In [27]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# data = "All work and no play makes jack a dull boy, all work and no play"
# print(word_tokenize(data))


In [79]:
# analyzed = analyzer.analyze(['мама', 'мыла', 'раму'])
%time analyzed = [list(analyzer.analyze(tokenized_sent_list)) for tokenized_sent_list in tokenized]

CPU times: user 1min 16s, sys: 96.7 ms, total: 1min 16s
Wall time: 1min 17s


In [90]:
analyzed[1]

[Morph(word='Смысл', lemma='смысл', tag=Tag(pos=NOUN,animacy=Inan,case=Nom,gender=Masc,number=Sing)),
 Morph(word='жизни', lemma='жизнь', tag=Tag(pos=NOUN,animacy=Inan,case=Gen,gender=Fem,number=Sing)),
 Morph(word='Где', lemma='где', tag=Tag(pos=ADV,degree=Pos)),
 Morph(word='найти', lemma='найти', tag=Tag(pos=VERB,aspect=Perf,verbform=Inf)),
 Morph(word='смысл', lemma='смысл', tag=Tag(pos=NOUN,animacy=Inan,case=Acc,gender=Masc,number=Sing)),
 Morph(word='существования', lemma='существование', tag=Tag(pos=NOUN,animacy=Inan,case=Gen,gender=Neut,number=Sing)),
 Morph(word='•', lemma='•', tag=Tag(pos=PUNCT)),
 Morph(word='Форум', lemma='форум', tag=Tag(pos=NOUN,animacy=Inan,case=Nom,gender=Masc,number=Sing)),
 Morph(word='сайта', lemma='сайт', tag=Tag(pos=NOUN,animacy=Inan,case=Gen,gender=Masc,number=Sing)),
 Morph(word='Pobedish.ru', lemma='pobedish.ru', tag=Tag(pos=UNKN))]

In [100]:
lemmatized_titles_with_nltk_and_maru = [' '.join([morph.lemma for morph in analyzed[i] if morph.tag.pos != 'PUNCT']) for i, m in enumerate(analyzed)] 

RangeIndex(start=0, stop=28026, step=1)

In [138]:
import numpy as np
from pandas import Series
df_titles['lemmatized_titles_with_nltk_and_maru'] = Series(np.array(lemmatized_titles_with_nltk_and_maru))

In [139]:
df_titles

,doc_id,title,lemmatized_titles_with_nltk_and_maru
0,26175,11.05.2015 4-й км Объездной дороги между Як-Бо...,11.05.2015 4-й километр объездной дорога як-бо...
1,26997,Смысл жизни или Где найти смысл существовани...,смысл жизнь где найти смысл существование фору...
2,1825,Как снизить холестерин в крови в домашних усло...,как снизить холестерин кровь домашний условие ...
3,12397,Как добраться до Крыма,как добраться крым
4,14089,Сообщество Steam :: Руководство :: Rust - Созд...,сообщество steam руководство rust создание нас...
5,22104,Обзор Tobii Eye Tracker 4C - ProGamer.Ru,обзор tobii eye tracker 4c progamer.ru
6,15231,"Отечественные, Наши фильмы - Художественные Ф...",отечественный наш фильм художественный фильм о...
7,5366,Бытовая техника для кухни. Столин. Фото и цены...,бытовой техника кухня столин фото цена бытовой...
8,3019,"Письмо, его виды, образец и правила оформления...",письмо вид образец правило оформление сопровод...
9,22151,Как избавиться от пауков в квартире?,как избавиться паук квартира


In [140]:
df_titles.to_csv('titles_and_lemmatized_titles_with_nltk_and_maru.csv')

In [141]:
df_test = pd.read_csv("titles_and_lemmatized_titles_with_nltk_and_maru.csv")

In [142]:
df_test

,Unnamed: 0,doc_id,title,lemmatized_titles_with_nltk_and_maru
0,0,26175,11.05.2015 4-й км Объездной дороги между Як-Бо...,11.05.2015 4-й километр объездной дорога як-бо...
1,1,26997,Смысл жизни или Где найти смысл существовани...,смысл жизнь где найти смысл существование фору...
2,2,1825,Как снизить холестерин в крови в домашних усло...,как снизить холестерин кровь домашний условие ...
3,3,12397,Как добраться до Крыма,как добраться крым
4,4,14089,Сообщество Steam :: Руководство :: Rust - Созд...,сообщество steam руководство rust создание нас...
5,5,22104,Обзор Tobii Eye Tracker 4C - ProGamer.Ru,обзор tobii eye tracker 4c progamer.ru
6,6,15231,"Отечественные, Наши фильмы - Художественные Ф...",отечественный наш фильм художественный фильм о...
7,7,5366,Бытовая техника для кухни. Столин. Фото и цены...,бытовой техника кухня столин фото цена бытовой...
8,8,3019,"Письмо, его виды, образец и правила оформления...",письмо вид образец правило оформление сопровод...
9,9,22151,Как избавиться от пауков в квартире?,как избавиться паук квартира
